# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the CIA World Factbook,
a compendium of statistics about all of the countries on Earth.

## Table of Contents

> #### 1. Introduction
> #### 2. Connect to database file
> #### 3. Explore the database
> #### 4. Compute summary statistics
> #### 5. Exploring Outliers
> #### 6. Exploring Average Population and Area
> #### 7. Finding Densely Populated Countries
> #### 8. Countries with higher death rate than birth rate
> #### 9. Conclusion

### Introduction

The `Goal of this project` is to `use SQL to analyze data from this database.`

The Factbook contains demographic information like the following:
* `population` — the global population.
* `population_growth` — the annual population growth rate, as a percentage.
* `area` — the total land and water area.

### Connect to database file

connect our Jupyter Notebook to our database file.
Make sure `%%capture` is the first line in the cell.

In [13]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

### Explore the database

#### Get info about the tables in the database

In [14]:
%%sql
SELECT *
FROM sqlite_master
WHERE type="table";

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We observe that the table we are dealing with is called the `facts` table.
Now we proceed to explore this table.

#### Get first 5 rows of the facts table

In [15]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* `name` — the name of the country.
* `area`— the country's total area (both land and water).
* `area_land` — the country's land area in square kilometers.
* `area_water` — the country's water area in square kilometers.
* `population` — the country's population.
* `population_growth` — the country's population growth as a percentage.
* `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
* `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

----

### Compute summary statistics

In [16]:
%%sql
SELECT MIN(population) AS min_population,
        MAX(population) AS max_population,
        MIN(population_growth) AS min_pop_growth, 
        MAX(population_growth) AS max_pop_growth
    FROM facts;

 * sqlite:///factbook.db
Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


From the result, we obsever the minimum population is 0 likewise the minimum population growth
The maximum population and maximun population growth however stands at `7,256,490,011` and `4.02` respectively

This therefore mean there is a country with a population of 0, 
as well as there is another country with a population of over 7.2 billion people

----

### Exploring Outliers

Here we try to get information regarding:
* Country with minimum population 
* Countries with maximum population

In [17]:
%%sql
SELECT *
    FROM facts
    WHERE population = ( SELECT MIN(population)
                        FROM facts
                        );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [18]:
%%sql
SELECT *
    FROM facts
    WHERE population = ( SELECT MAX(population)
                        FROM facts
                        );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Our result makes sense as `Antartctica` is unhabitted by humans, thus a population of 0.
We also notice that the popultion of over 7.2 billion we got above was the population of the whole world, not a specific country.

---

### Exploring Average Population and Area

#### Recompute the summary statistics excluding the World

In [19]:
%%sql
SELECT MIN(population) AS min_population,
        MAX(population) AS max_population,
        MIN(population_growth) AS min_pop_growth, 
        MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE name != "World";

 * sqlite:///factbook.db
Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


We now discover that there is a country with a population of `over 1.36 billion people`

#### calculate the average value for population and area columns
To do this correctly, we have to exclude the World row as it will give a wrong result by projecting the average up

In [20]:
%%sql
SELECT AVG(population) AS avg_population,
       AVG(area) AS avg_area
    FROM facts
    WHERE name != "World";

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32242666.56846473,555093.546184739


### Finding Densely Populated Countries
In this section, we will identify countries that have the following:

* Above-average values for population.
* Below-average values for area.

In [21]:
%%sql
SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population)
                        FROM facts
                        WHERE name != "World");

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
28,bm,Burma,676578,653508,23070,56320206,1.01,18.39,7.96,0.28
32,ca,Canada,9984670,9093507,891163,35099836,0.75,10.28,8.42,5.66
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
38,co,Colombia,1138910,1038700,100210,46736728,1.04,16.47,5.4,0.64
40,cg,"Congo, Democratic Republic of the",2344858,2267048,77810,79375136,2.45,34.88,10.07,0.27


In [22]:
%%sql
SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population)
                        FROM facts
                        WHERE name != "World")
            AND area < (SELECT AVG(area)
                        FROM facts
                        WHERE name != "World");

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


### Countries with higher death rate than birth rate

In [23]:
%%sql
SELECT *
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY death_rate DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
116,md,Moldova,33851,32891,960,3546847,1.03,12.0,12.59,9.67
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6


### Conclusion
From the last section, we conclude that Ukrain, Bulgaria and Latvia has the highest death rate.
Coupled with this, their death rate is higher than birth rate.